In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [ ]:
import sys
sys.path.insert(0, "python")
from network import *
from protocols import *
from adversary import *
from simulator import *

In [ ]:
def shorten_protocol_name(x: str):
    val = x.replace("Protocol","").replace("spreading_proba","p")
    val = val.split("broadcast")[0][:-1].replace("(", ": ")
    if val[-1] == ",":
        val = val[:-1]
    return val

In [ ]:
nw_generator = NodeWeightGenerator("random")
ew_generator = EdgeWeightGenerator("normal")

# Dandelion example

In [ ]:
net = Network(nw_generator, ew_generator, num_nodes=1000, k=20)

In [ ]:
dp = DandelionProtocol(net, 0.5)

dp = DandelionPlusPlusProtocol(net, 0.5)

nx.draw(dp.anonymity_graph, node_size=20)

In [ ]:
%%time
adv = Adversary(net, 0.1)
sim = Simulator(dp, adv, 30, verbose=False)
sim.run()

In [ ]:
%%time
evaluator = Evaluator(sim, "first_reach")
print(evaluator.get_report())

In [ ]:
%%time
evaluator = Evaluator(sim, "first_sent")
print(evaluator.get_report())

In [ ]:
%%time
evaluator = Evaluator(sim, "dummy")
print(evaluator.get_report())

# Spreading time experiment

In [ ]:
%%time
bc_mode="sqrt"
net = Network(nw_generator, ew_generator, num_nodes=100, k=10)
print(net.num_nodes)
protocols = [
    DandelionProtocol(net, 0.5, broadcast_mode=bc_mode),
    DandelionProtocol(net, 0.25, broadcast_mode=bc_mode),
    DandelionPlusPlusProtocol(net, 0.5, broadcast_mode=bc_mode),
    DandelionPlusPlusProtocol(net, 0.25, broadcast_mode=bc_mode),
    BroadcastProtocol(net, broadcast_mode=bc_mode)
]

for protocol in protocols:
    sim = Simulator(protocol, adv, 10, verbose=False)
    msg_cov = sim.run(coverage_threshold=1.0)
    #print(msg_cov)
    print(np.mean(msg_cov))
    x = np.arange(0.1,1.0,0.1)
    mean_quantiles, std_quantiles = sim.node_contact_time_quantiles(x)
    name = shorten_protocol_name(str(protocol)) if not "Broadcast" in str(protocol) else str(protocol)
    plt.plot(x, mean_quantiles, label=name)
plt.legend()
plt.ylabel("time (ms)")
plt.xlabel("quantiles")

# Experiment example

In [ ]:
network_size = 1000
config = {
    "network_size" : network_size,
    "num_msg" : int(network_size*0.05),
    "degree" : 50,
    "bc_mode" : "sqrt",
    "dandelion_bc_probas" : [0.5, 0.25, 0.1]
}
adversary_ratios = [0.01,0.05,0.1,0.2]
num_trials = 2
max_threads = 2

In [ ]:
print(config)

In [ ]:
from tqdm.notebook import tqdm

### Messages are propagated until full coverage

In [ ]:
import os
print(os.cpu_count())

In [ ]:
def run_and_eval(net: Network, adv: Adversary, protocol: Protocol, num_msg: int):
    sim = Simulator(protocol, adv, num_msg, verbose=False)
    sim.run(coverage_threshold=1.0)
    reports = []
    for estimator in ["first_reach","first_sent"]:
        evaluator = Evaluator(sim, estimator)
        report = evaluator.get_report()
        report["protocol"] = str(protocol)
        report["adversary_ratio"] = adv.ratio
        reports.append(report)
    return reports, sim

In [ ]:
def run_single_experiment(config):
    net = Network(nw_generator, ew_generator, config["network_size"], config["degree"])
    adv = Adversary(net, config["adv_ratio"], config["is_active_adversary"])
    protocols = [BroadcastProtocol(net, broadcast_mode=config["bc_mode"])]
    for broadcast_proba in config["dandelion_bc_probas"]:
        protocols.append(DandelionProtocol(net, broadcast_proba, broadcast_mode=config["bc_mode"]))
        protocols.append(DandelionPlusPlusProtocol(net, broadcast_proba, broadcast_mode=config["bc_mode"]))
    single_run_results = []
    for protocol in protocols:
        new_reports, sim = run_and_eval(net, adv, protocol, config["num_msg"])
        single_run_results += new_reports
    print(config["adv_ratio"])
    return single_run_results

In [ ]:
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor

In [ ]:
def run_experiment(adversary_ratios, num_trials, config, is_active_adversary=False, max_workers=1):
    executor = ThreadPoolExecutor(max_workers=max_workers)
    queries = []
    for adv_ratio in adversary_ratios*num_trials:
        query = config.copy()
        query["adv_ratio"] = adv_ratio
        query["is_active_adversary"] = is_active_adversary
        queries.append(query)
    #print(queries)
    pool = executor.map(run_single_experiment, queries)
    executor.shutdown()
    results = []
    for res in pool:
        results += res
    results_df = pd.DataFrame(results)
    return results_df

In [ ]:
%%time
results_df = run_experiment(adversary_ratios, num_trials, config, False, max_threads)

In [ ]:
len(results_df)

In [ ]:
def shorten_protocol_names_for_df(df, col="protocol"):
    tmp_df = df.copy()
    tmp_df[col] = tmp_df[col].apply(shorten_protocol_name)
    return tmp_df

In [ ]:
results_df_short = shorten_protocol_names_for_df(results_df)

In [ ]:
fr_results_df = results_df_short[results_df_short["estimator"]=="first_reach"]
fs_results_df = results_df_short[results_df_short["estimator"]=="first_sent"]
print(fr_results_df.shape, fs_results_df.shape)

## a.) Adversary performance

- First-sent estimator performs better than first reach - **OK**

In [ ]:
fr_results_df_short = shorten_protocol_names_for_df(fr_results_df)
fs_results_df_short = shorten_protocol_names_for_df(fs_results_df)

In [ ]:
fig, ax = plt.subplots(3,2, figsize=(20,18))
sns.boxplot(data=fr_results_df, x="adversary_ratio", y="hit_ratio", hue="protocol", ax=ax[0][0])
sns.boxplot(data=fr_results_df, x="adversary_ratio", y="inverse_rank", hue="protocol", ax=ax[1][0])
sns.boxplot(data=fr_results_df, x="adversary_ratio", y="ndcg", hue="protocol", ax=ax[2][0])
sns.boxplot(data=fs_results_df, x="adversary_ratio", y="hit_ratio", hue="protocol", ax=ax[0][1])
sns.boxplot(data=fs_results_df, x="adversary_ratio", y="inverse_rank", hue="protocol", ax=ax[1][1])
sns.boxplot(data=fs_results_df, x="adversary_ratio", y="ndcg", hue="protocol", ax=ax[2][1])
for i in range(3):
    for j in range(2):
        ax[i][j].set_title("First-reach estimator" if j % 2 == 0 else "First-sent estimator")
        ax[i][j].set_ylim(0,1.0)

While the fraction of nodes reached by messages are the same range for all protocols:
- actually, almost all nodes are reached by the messages (due to `coverage_threshold=1.0` setting) - **OK**

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(20,6))
sns.boxplot(data=results_df_short, x="adversary_ratio", y="message_spread_ratio", hue="protocol", ax=ax)

## b.) Passive vs. active adversary

**active adversary:** simply does not forward the received message

**passive adversary:** forwards every message according to protocol rules

In [ ]:
%%time
active_results_df = run_experiment(adversary_ratios, num_trials, config, False, max_threads)
active_results_df = active_results_df[active_results_df["estimator"]=="first_sent"]
active_results_df.shape

In [ ]:
active_results_df_short = shorten_protocol_names_for_df(active_results_df)

The results show that **longer stem-phase in the Dandelion protocol** (low broadcast probability) **have a negative impact on the message spread ratio** (ratio of all nodes who receives the message) - as expected.

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(20,6))
sns.boxplot(data=results_df_short, x="adversary_ratio", y="message_spread_ratio", hue="protocol", ax=ax[0])
ax[0].set_title("Passive adversary")
ax[0].set_ylim(0.1,1.0)
sns.boxplot(data=active_results_df_short, x="adversary_ratio", y="message_spread_ratio", hue="protocol", ax=ax[1])
ax[1].set_title("Active adversary")
ax[1].set_ylim(0.1,1.0)

In [ ]:
import sys, os, pytest

sys.path.insert(0, "%s/python" % os.getcwd())
import networkx as nx
from network import Network, NodeWeightGenerator, EdgeWeightGenerator
from simulator import Simulator, Evaluator
from message import Message
from protocols import BroadcastProtocol, DandelionProtocol, DandelionPlusPlusProtocol
from adversary import Adversary

In [ ]:
### First sent vs First reach test
rnd_node_weight = NodeWeightGenerator("random")
rnd_edge_weight = EdgeWeightGenerator("random")
G = nx.DiGraph()
G.add_nodes_from([1, 2, 3])
G.add_weighted_edges_from([(1, 2, 0.9), (1, 3, 1.84),(2, 3, 1.01)], weight="latency")
print(nx.draw(G,with_labels=True))
net = Network(rnd_node_weight, EdgeWeightGenerator("custom"), graph=G)
protocol = BroadcastProtocol(net, seed=44)
adv = Adversary(net, ratio=0.0, adversaries=[2])
print(adv.nodes)
assert 3 in adv.nodes

# start a message from Node 1
msg = Message(1)
receiver_order = [1, 2, 2, 2]
for i, receiver in enumerate(receiver_order):
    msg.process(protocol, adv)
    print(i,receiver,msg.history)
    assert receiver in msg.history
    # assert len(msg.history) == i + 1

In [ ]:
sim = Simulator(protocol, adv, 1, True)
sim.run(1.0)
for estimator in ["first_reach", "first_sent"]:
    evaluator = Evaluator(sim, estimator)
    results = [
        evaluator.exact_hits,
        evaluator.ranks,
        evaluator.inverse_ranks,
        evaluator.entropies,
    ]
    for i in results:
        print(estimator, i)